In [1]:
from ta.momentum import PercentagePriceOscillator,RSIIndicator,ROCIndicator,WilliamsRIndicator,StochRSIIndicator
from ta.trend import MACD,ADXIndicator,CCIIndicator,sma_indicator,ema_indicator,PSARIndicator
from ta.volatility import BollingerBands,AverageTrueRange
from ta.volume import OnBalanceVolumeIndicator,ChaikinMoneyFlowIndicator,MFIIndicator
import warnings
import plotly.express as px
warnings.simplefilter('ignore')
import numpy as np
import plotly.io as pio
pio.renderers.default = "notebook"
import pandas as pd
from binance import Client
api_key="LmId7IJ05Pft6aR9JtGhmlauJ9DfTHtH9BhWIdbYPkCBowbwejnLCExCR93gqhTT"
secret_key="MUuJx69AafPeVGbO5yuCEhSULacvxJm0lWlrVZJAa6Il1ZcIkgznt9Ru7JtifUPb"
import seaborn as sns
import matplotlib.pyplot as plt
def get_binance_data(api_key, pair, countdown='open', interval='4h', start='6 December 2019', end='6 December 2022'):
  client = Client(api_key=api_key)
  intervals = {
      '1h':  Client.KLINE_INTERVAL_1HOUR
  }
  interval = intervals.get(interval, '1h')
  print(f'Historical interval {interval}')
  klines = client.get_historical_klines(symbol=pair, interval=interval, start_str=start, end_str=end)
  data = pd.DataFrame(klines)
  data.columns = ['open_time','open', 'high', 'low', 'close', 'volume','close_time', 'qav','num_trades','taker_base_vol','taker_quote_vol', 'ignore']
  data.index = [pd.to_datetime(x, unit='ms').strftime('%Y-%m-%d %H:%M:%S') for x in data.open_time]
  usecols=['open', 'high', 'low', 'close', 'volume']
  data = data[usecols]
  data = data.astype('float')
  return data
from scipy.stats import linregress
def momentum(closes):
    returns = np.log(closes)
    x = np.arange(len(returns))
    slope, _, rvalue, _, _ = linregress(x, returns)
    return ((1 + slope) ** 252) * (rvalue ** 2)  # annualize slope and multiply by R^2
#ppo
def get_features(btc,fill=False):
    indicator_ppo = PercentagePriceOscillator(close=btc["close"],fillna=fill)
    indicator_rsi= RSIIndicator(close=btc['close'],fillna=fill)
    indicator_adx= ADXIndicator(high=btc['high'],low=btc['low'],close=btc['close'],fillna=fill)
    indicator_cci=CCIIndicator(high=btc['high'],low=btc['low'],close=btc['close'],fillna=fill)
    indicator_roc = ROCIndicator(close=btc['close'],fillna=fill)
    indicator_macd=MACD(close=btc['close'],fillna=fill)
    indicator_william_r = WilliamsRIndicator(high=btc['high'],low=btc['low'],close=btc['close'],fillna=fill)
    indicator_bollinger= BollingerBands(close=btc['close'],fillna=fill)
    indicator_psa_r = PSARIndicator(high=btc['high'],low=btc['low'],close=btc['close'],fillna=fill)
    indicator_obv=OnBalanceVolumeIndicator(close=btc['close'],volume=btc['volume'],fillna=fill)
    indicator_chaikin=ChaikinMoneyFlowIndicator(high=btc['high'],low=btc['low'],close=btc['close'],volume=btc['volume'],fillna=fill)
    indicator_mfi= MFIIndicator(high=btc['high'],low=btc['low'],close=btc['close'],volume=btc['volume'],fillna=fill)
    indicator_ATR = AverageTrueRange(high=btc['high'],low=btc['low'],close=btc['close'],fillna=fill)
    indicator_stoch=StochRSIIndicator(close=btc['close'],fillna=fill)
    btc['rsi']=indicator_rsi.rsi()

    btc['ppo']=indicator_ppo.ppo()
    btc['macd']=indicator_macd.macd()
    btc['macd_signal']=indicator_macd.macd_signal()
    btc['macd_histogram']=indicator_macd.macd_diff()

    btc['adx']=indicator_adx.adx()
    btc['cci']=indicator_cci.cci()
    btc['roc']=indicator_roc.roc()
    btc['william_r']=indicator_william_r.williams_r()
    btc['sma20']=sma_indicator(close=btc['close'],window=20,fillna=fill)
    btc['sma50']=sma_indicator(close=btc['close'],window=50,fillna=fill)
    btc['sma100']=sma_indicator(close=btc['close'],window=100,fillna=fill)
    btc['ema20']=ema_indicator(close=btc['close'],window=20,fillna=fill)
    btc['ema50']=ema_indicator(close=btc['close'],window=50,fillna=fill)
    btc['ema100']=ema_indicator(close=btc['close'],window=100,fillna=fill)
    btc['bollinger_high']=indicator_bollinger.bollinger_hband()
    btc['bollinger_low']=indicator_bollinger.bollinger_lband()
    btc['bollinger_pband']=indicator_bollinger.bollinger_pband()
    btc['psa_r']=indicator_psa_r.psar()
    btc['obv']=indicator_obv.on_balance_volume()
    btc['cmf']=indicator_chaikin.chaikin_money_flow()
    btc['mfi']=indicator_mfi.money_flow_index()
    btc['atr']=indicator_ATR.average_true_range()
    btc[r'%d']=indicator_stoch.stochrsi_d()
    btc[r'%k']=indicator_stoch.stochrsi_k()
    btc['momentum']=btc['close'].rolling(90).apply(momentum,raw=False)
    if fill:
        btc['momentum']=btc['momentum'].fillna(btc['momentum'].mean())
    return btc

In [2]:
# not here I am filling the NA values as per the TA as NA can not be used in ML models, so setting fill=True
btc=get_binance_data(api_key=api_key,pair="BTCBUSD")
btc=get_features(btc=btc,fill=True)
eth=get_binance_data(api_key=api_key,pair="ETHBUSD")
eth=get_features(btc=eth,fill=True)
bnb=get_binance_data(api_key=api_key,pair="BNBBUSD")
bnb=get_features(btc=bnb,fill=True)

Historical interval 1d
Historical interval 1d
Historical interval 1d


### Preprocessing

#### Creating the target variable based on  Target Formulation (III.B)

In [3]:
# first compute the difference (if change is postive then 1 otherwise 0).. diff() computes the change
def target_formulation(close):
    return close.diff().apply(lambda x: 1 if x>0 else 0)

In [4]:
btc['y']=target_formulation(btc['close'])
eth['y']=target_formulation(eth['close'])
bnb['y']=target_formulation(bnb['close'])

In [5]:
# now apply preprocessing on variables other than target varable
# first get the independnt variables

btc_X= btc.drop('y',axis=1)
eth_X= eth.drop('y',axis=1)
bnb_X= bnb.drop('y',axis=1)



In [6]:
from pywt import wavedec
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding,SpectralEmbedding

In [22]:
# now putting the above analysis and operations in a function to get data for all pairs
def preprocess(data,dim_reduction:str='PCA'):
    scaler=MinMaxScaler()
    data_X=pd.DataFrame(scaler.fit_transform(data),columns=data.columns.values.tolist(),index=data.index.values.tolist())
    if dim_reduction=="PCA":
        pca=PCA(n_components=None,random_state=101)
        pca.fit(data)
        exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
        expl_top=pd.DataFrame(pd.Series(dict(zip(data_X.columns.values.tolist(),exp_var_cumul))),columns=['exp variance'])
        data_X=data_X.loc[:,expl_top[expl_top['exp variance']>=0.95].index.values.tolist()]
        coeffs=wavedec(data=data_X,wavelet='haar',level=2,mode='symmetric')
        return pd.DataFrame(np.array(coeffs[0]))
    if dim_reduction == 'SE':
        embedding = SpectralEmbedding(n_components=10)
        X_transformed = embedding.fit_transform(data)
        coeffs=wavedec(data=X_transformed,wavelet='haar',level=2,mode='symmetric')
        return pd.DataFrame(np.array(coeffs[0]))
    if dim_reduction=='LLE':
        embedding = LocallyLinearEmbedding(n_components=10)
        X_transformed = embedding.fit_transform(data)
        coeffs=wavedec(data=data_X,wavelet='haar',level=2,mode='symmetric')
        return pd.DataFrame(np.array(coeffs[0]))
        

        
    
                        



## 1. PCA

In [11]:

btc_X= btc.drop('y',axis=1)
eth_X= eth.drop('y',axis=1)
bnb_X= bnb.drop('y',axis=1)
btc_X=preprocess(btc_X,dim_reduction='PCA')
eth_X=preprocess(eth_X,dim_reduction='PCA')
bnb_X=preprocess(bnb_X,dim_reduction='PCA')

In [12]:
btc_y=btc['y']
eth_y=eth['y']
bnb_y=bnb['y']

## Multi Objective optimization using GA

In [13]:

import sklearn.model_selection
import sklearn.metrics
import sklearn.tree
from xgboost import XGBClassifier
import pymoo.util.nds.non_dominated_sorting as nds
import hiplot as hip

cv_objs = [
    'Mean CV Accuracy',
    'Mean CV True Positive Rate',
    'Mean CV False Positive Rate',
    'Mean CV AUC'
]
cv_objs_max = ['Mean CV Accuracy', 'Mean CV True Positive Rate', 'Mean CV AUC']
test_objs = [
    'Test Accuracy',
    'Test True Positive Rate',
    'Test False Positive Rate',
    'Test AUC'
]


def data_preparation(X,y):
    test_size = 0.25
    # Split
    X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(
            X,
            y,
            test_size=test_size,
            random_state=1008,
            stratify=y
        )
    return X_train, X_test, y_train, y_test


def default_hyperparameter(X_train, y_train):
    xgb = XGBClassifier( objective='binary:logistic')
    xgb.fit(X_train, y_train)
    return xgb


def single_objective_gridsearch(X_train, y_train):
    parameter_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
    
    gs = sklearn.model_selection.GridSearchCV(
        XGBClassifier(),
        parameter_grid,
        cv=5,
        scoring='accuracy',
        n_jobs=-1
    )
    gs.fit(X_train, y_train)
    clf =  XGBClassifier(
        min_child_weight=gs.best_params_['min_child_weight'],
        gamma=gs.best_params_['gamma'],
        subsample=gs.best_params_['subsample'],
        colsample_bytree=gs.best_params_['colsample_bytree'],
        max_depth=gs.best_params_['max_depth']

    )
    clf.fit(X_train, y_train)
    return clf, gs


def fpr(y_true, y_pred):
    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_true, y_pred).ravel()
    obj = fp / (fp + tn)
    return obj


def tpr(y_true, y_pred):
    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_true, y_pred).ravel()
    obj = tp / (tp + fn)
    return obj


def multi_objective_gridsearch(X_train, y_train):
    parameter_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
    }
    scoring = {
        'Accuracy': 'accuracy',
        'True Positive Rate': sklearn.metrics.make_scorer(tpr),
        'False Positive Rate': sklearn.metrics.make_scorer(fpr),
        'AUC': 'roc_auc'
    }
    gs = sklearn.model_selection.GridSearchCV(
        XGBClassifier(),
        parameter_grid,
        cv=5,
        scoring=scoring,
        n_jobs=-1,
        refit=False
    )
    gs.fit(X_train, y_train)
    df = pd.DataFrame(gs.cv_results_['params'])
    df['Mean CV Accuracy'] = gs.cv_results_['mean_test_Accuracy']
    df['Mean CV True Positive Rate'] = \
        gs.cv_results_['mean_test_True Positive Rate']
    df['Mean CV False Positive Rate'] = \
        gs.cv_results_['mean_test_False Positive Rate']
    df['Mean CV AUC'] = gs.cv_results_['mean_test_AUC']
    return df


def nondom_sort(df, objs, max_objs=None):
    df_sorting = df.copy()
    # Flip Objectives to Maximize
    if max_objs is not None:
        df_sorting[max_objs] = -1.0 * df_sorting[max_objs]
    # Non-dominated Sorting
    nondom_idx = nds.find_non_dominated(df_sorting[objs].values)
    return df.iloc[nondom_idx].copy()


def parallel_plot(df, color_column, invert_column):
    # Make Unique IDs
    df['Solution ID'] = df.index + 1
    df['Solution ID'] = df['Solution ID'].apply(lambda x: '{0:0>5}'.format(x))
    df['Solution ID'] = 'S'+df['Solution ID'].astype(str)
    # Create Plot
    exp = hip.Experiment.from_dataframe(df)
    exp.parameters_definition[color_column].colormap = 'interpolateViridis'
    exp.display_data(hip.Displays.PARALLEL_PLOT).update(
        {
            'hide': [
                'uid',
                'max_features',
                'min_samples_split',
                'Solution ID'
            ],
            'invert': invert_column
        }
    )
    exp.display_data(hip.Displays.TABLE).update({'hide': ['uid', 'from_uid']})
    return exp


def get_test_performance(X_train, X_test, y_train, y_test, params):
    # Fit Model with Specified Hyperparameters
    clf = XGBClassifier(
        min_child_weight=int(params['min_child_weight']),
        gamma=float(params['gamma']),
        subsample=float(params['subsample']),
        colsample_bytree=float(params['colsample_bytree']),
        max_depth=int(params['max_depth'])

       
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    # Compute Objectives on Test Set
    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, y_pred).ravel()
    acc = (tp + tn) / (tn + fp + fn + tp)
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)
    auc = sklearn.metrics.roc_auc_score(
        y_test,
        clf.predict_proba(X_test)[:, 1]
    )
    return pd.Series([acc, tpr, fpr, auc], test_objs)

accuracies=[]
    # Prepare
def main(X,y,currency,dim_reduction:str='PCA'):
    X_train, X_test, y_train, y_test = data_preparation(X,y)
    # Default Hyperparameter Values
    clf_default = default_hyperparameter(X_train, y_train)
    print(clf_default.get_params())
    print('Train Accuracy:', sklearn.metrics.accuracy_score(
        y_train, clf_default.predict(X_train))
    )
    
    print('Test Accuracy:', sklearn.metrics.accuracy_score(
        y_test, clf_default.predict(X_test))
    )
    accuracies.append({"dim_reduction":dim_reduction,
                      "train_accuracy":sklearn.metrics.accuracy_score(
        y_train, clf_default.predict(X_train)),
                      "test_accuracy":sklearn.metrics.accuracy_score(
        y_test, clf_default.predict(X_test)),
                      "currency":currency})
    # Single Objective Grid Search
    clf_SO, gs_SO = single_objective_gridsearch(X_train, y_train)
    print(clf_SO.get_params())
    print('CV Train Accuracy:', gs_SO.best_score_)
    print('Test Accuracy:', sklearn.metrics.accuracy_score(
        y_test, gs_SO.predict(X_test))
    )
    # Multi-Objective Grid Search
    df_all = multi_objective_gridsearch(X_train, y_train)
    parallel_plot(
        df_all,
        color_column='Mean CV Accuracy',
        invert_column=cv_objs_max
    ).to_html(f'{currency}_all.html')
    df_non_dom = nondom_sort(df_all, cv_objs, max_objs=cv_objs_max)
    parallel_plot(
        df_non_dom,
        color_column='Mean CV Accuracy',
        invert_column=cv_objs_max
    ).to_html(f'{currency}_{dim_reduction}non_dom.html')
    # Non-Dominated Set Test Performance
    df_non_dom_test = df_non_dom.apply(
        lambda row: get_test_performance(
            X_train, X_test, y_train, y_test, row
        ),
        axis=1
    )
    df_non_dom = df_non_dom.join(df_non_dom_test)
    # Check if Performance is Preserved by Looking at Sorted Objective Values
    for i, j in zip(cv_objs, test_objs):
        print(df_non_dom[[i, j]].sort_values(i, ascending=False))
    



In [14]:
main(btc_X,btc_y,currency="BTC",dim_reduction="PCA")

{'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': 0.5, 'booster': 'gbtree', 'callbacks': None, 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'gamma': 0, 'gpu_id': -1, 'grow_policy': 'depthwise', 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.300000012, 'max_bin': 256, 'max_cat_to_onehot': 4, 'max_delta_step': 0, 'max_depth': 6, 'max_leaves': 0, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 100, 'n_jobs': 0, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'sampling_method': 'uniform', 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'exact', 'validate_parameters': 1, 'verbosity': None}
Train Accuracy: 1.0
Test Accuracy: 0.610909090909091
{'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': 0.5, 'booster': 'gbtree', 'ca

In [15]:
main(eth_X,eth_y,currency="ETH",dim_reduction="PCA")

{'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': 0.5, 'booster': 'gbtree', 'callbacks': None, 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'gamma': 0, 'gpu_id': -1, 'grow_policy': 'depthwise', 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.300000012, 'max_bin': 256, 'max_cat_to_onehot': 4, 'max_delta_step': 0, 'max_depth': 6, 'max_leaves': 0, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 100, 'n_jobs': 0, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'sampling_method': 'uniform', 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'exact', 'validate_parameters': 1, 'verbosity': None}
Train Accuracy: 1.0
Test Accuracy: 0.6472727272727272
{'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': 0.5, 'booster': 'gbtree', 'c

In [16]:
main(bnb_X,bnb_y,currency="BNB",dim_reduction="PCA")

{'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': 0.5, 'booster': 'gbtree', 'callbacks': None, 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'gamma': 0, 'gpu_id': -1, 'grow_policy': 'depthwise', 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.300000012, 'max_bin': 256, 'max_cat_to_onehot': 4, 'max_delta_step': 0, 'max_depth': 6, 'max_leaves': 0, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 100, 'n_jobs': 0, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'sampling_method': 'uniform', 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'exact', 'validate_parameters': 1, 'verbosity': None}
Train Accuracy: 1.0
Test Accuracy: 0.6727272727272727
{'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': 0.5, 'booster': 'gbtree', 'c

In [17]:

btc_X= btc.drop('y',axis=1)
eth_X= eth.drop('y',axis=1)
bnb_X= bnb.drop('y',axis=1)
btc_X=preprocess(btc_X,dim_reduction='SE')
eth_X=preprocess(eth_X,dim_reduction='SE')
bnb_X=preprocess(bnb_X,dim_reduction='SE')
main(btc_X,btc_y,currency="BTC",dim_reduction="SE")
main(eth_X,eth_y,currency="ETH",dim_reduction="SE")
main(bnb_X,bnb_y,currency="BNB",dim_reduction="SE")

{'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': 0.5, 'booster': 'gbtree', 'callbacks': None, 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'gamma': 0, 'gpu_id': -1, 'grow_policy': 'depthwise', 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.300000012, 'max_bin': 256, 'max_cat_to_onehot': 4, 'max_delta_step': 0, 'max_depth': 6, 'max_leaves': 0, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 100, 'n_jobs': 0, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'sampling_method': 'uniform', 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'exact', 'validate_parameters': 1, 'verbosity': None}
Train Accuracy: 0.9890510948905109
Test Accuracy: 0.48
{'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': 0.5, 'booster': 'gbtree', '

In [24]:

btc_X= btc.drop('y',axis=1)
eth_X= eth.drop('y',axis=1)
bnb_X= bnb.drop('y',axis=1)
btc_X=preprocess(btc_X,dim_reduction='LLE')

eth_X=preprocess(eth_X,dim_reduction='LLE')
bnb_X=preprocess(bnb_X,dim_reduction='LLE')
main(btc_X,btc_y,currency="BTC",dim_reduction="LLE")
main(eth_X,eth_y,currency="ETH",dim_reduction="LLE")
main(bnb_X,bnb_y,currency="BNB",dim_reduction="LLE")

{'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': 0.5, 'booster': 'gbtree', 'callbacks': None, 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'gamma': 0, 'gpu_id': -1, 'grow_policy': 'depthwise', 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.300000012, 'max_bin': 256, 'max_cat_to_onehot': 4, 'max_delta_step': 0, 'max_depth': 6, 'max_leaves': 0, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 100, 'n_jobs': 0, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'sampling_method': 'uniform', 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'exact', 'validate_parameters': 1, 'verbosity': None}
Train Accuracy: 1.0
Test Accuracy: 0.610909090909091
{'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': 0.5, 'booster': 'gbtree', 'ca

In [26]:
pd.DataFrame(accuracies).round(3)

,dim_reduction,train_accuracy,test_accuracy,currency
0,PCA,1.000,0.611,BTC
1,PCA,1.000,0.647,ETH
2,PCA,1.000,0.673,BNB
3,SE,0.989,0.480,BTC
4,SE,0.990,0.498,ETH
5,SE,0.989,0.505,BNB
6,LLE,1.000,0.611,BTC
7,LLE,1.000,0.647,ETH
8,LLE,1.000,0.673,BNB
